In [3]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import os
from urllib.parse import urljoin
import http.server
import socketserver
import webbrowser
from urllib.parse import unquote

# 데이터베이스 초기화 함수
DATABASE = 'news_db.sqlite'

# 데이터베이스 초기화 및 테이블 생성
def init_db():
    with sqlite3.connect(DATABASE) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS news (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                url TEXT UNIQUE,
                title TEXT,
                views INTEGER DEFAULT 0
            )
        ''')
        conn.commit()

# 네이버 뉴스 사이트에 접속
url = 'https://news.naver.com/'
response = requests.get(url)

# 최신 뉴스 기사 제목과 링크를 추출
soup = BeautifulSoup(response.text, 'html.parser')
articles = soup.find_all('li', class_='cnf_news_item')  # 실제 클래스명을 네이버 뉴스 사이트의 클래스명으로 바꾸세요.

# 데이터베이스 초기화 (테이블 생성)
init_db()

# 추출된 데이터를 SQLite에 저장
def insert_or_update_article(title, link):
    with sqlite3.connect(DATABASE) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            INSERT OR REPLACE INTO news (url, title, views) VALUES (?, ?, 0)
        ''', (link, title))
        conn.commit()

for article in articles:
    title = article.get_text().strip()
    link = article.find('a')['href']
    full_link = urljoin(url, link)
    insert_or_update_article(title, full_link)

# HTML 파일 생성
html_content = '''<html>
<head>
    <meta charset="UTF-8">  <!-- UTF-8 문자 인코딩 설정 -->
    <title>네이버 뉴스 최신 기사 목록</title>
</head>
<body>
    <h1>네이버 뉴스 최신 기사 목록</h1>
    <ul>'''

with sqlite3.connect(DATABASE) as conn:
    cursor = conn.cursor()
    cursor.execute('SELECT id, title, url, views FROM news')
    articles = cursor.fetchall()

    for article in articles:
        html_content += f'<li><a href="/view/{article[0]}" target="_blank">{article[1]}</a> - 조회수: {article[3]}</li>'

html_content += '''</ul>
</body>
</html>'''

# HTML 파일로 저장
file_path = 'news_list.html'
with open(file_path, 'w', encoding='utf-8') as f:
    f.write(html_content)

# 조회수 증가를 위한 핸들러 함수
class MyHandler(http.server.SimpleHTTPRequestHandler):
    def do_GET(self):
        if self.path.startswith('/view/'):
            article_id = self.path.split('/view/')[1]
            with sqlite3.connect(DATABASE) as conn:
                cursor = conn.cursor()
                cursor.execute('SELECT url FROM news WHERE id = ?', (article_id,))
                article = cursor.fetchone()

                if article:
                    cursor.execute('UPDATE news SET views = views + 1 WHERE id = ?', (article_id,))
                    conn.commit()
            
            # 조회수 업데이트 후 해당 뉴스 페이지로 리다이렉션
            self.send_response(303)
            self.send_header('Location', article[0])  # 리다이렉션할 URL
            self.end_headers()
            return
        
        # 기본 GET 요청 처리 (파일 요청)
        return super().do_GET()

# 8000번 포트로 HTTP 서버 실행
PORT = 8000
Handler = MyHandler
httpd = socketserver.TCPServer(("", PORT), Handler)
print(f"Server started at http://localhost:{PORT}")
webbrowser.open(f'http://localhost:{PORT}/{file_path}')  # 웹 브라우저에서 자동으로 열기
httpd.serve_forever()


OSError: [WinError 10048] 각 소켓 주소(프로토콜/네트워크 주소/포트)는 하나만 사용할 수 있습니다